<a href="https://colab.research.google.com/github/Krati012/IDP-2020/blob/master/TRRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive/My Drive/CTW_2019"

 AE_Model_Conv2.h5	  CTW2019_logarithimic_mean_sqr.h5
 AE_Model_Conv.h5	  CTW2019_mean_sqr2.h5
 AutoEncoderModel1.h5	  CTW2019_mean_sqr.h5
 AutoEncoderModel2.h5	  data2019.h5
 AutoEncoderModel.h5	  data_feat.h5
 best_AE1.h5		  h_Estimated_CTW_Train.h5
 best_AE2.h5		  Histogram_MSE.png
 best_AE3.h5		  Histogram.png
 best_AE4.h5		  Histogram_RMSE.png
 best_cnn.h5		  NDR.ipynb
 best_cnn_ndr2.h5	 'PCA(1056)+CNN'
 best_cnn_ndr3.h5	 'PCA(66)+CNN'
 best_cnn_ndr4.h5	  PCA+MLP_Hist.png
 best_cnn_ndr5_adam.h5	  PCA+MLP.ipynb
 best_cnn_ndr.h5	  PCA_Unsupervised.sav
 best.h5		  r_Position_CTW_Train.h5
 Best_PCA_CNN_2.h5	  saved_model.pb
 Best_PCA_CNN.h5	  SimpleAE.h5
 best_PCA.h5		  SimpleAE_pred.h5
'Copy of AE.ipynb'	  SNR_CTW_Train.h5
'Copy of PCA+CNN'	  test.out
 CTW2019_Dataset_h5.zip


In [3]:
import numpy as np # linear algebra
import keras
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [4]:
'''#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)


batch_size = 30
epochs = 50
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cnn_training.h5'
'''

"#getting and processing data\n#h_Estimated   ---> shape = (2, 924, 16, 17486)\n#r_postition ---> shape = (3, 17486)\n#SNR_Est    ---> shape = (16, 17486)\n\n\nbatch_size = 30\nepochs = 50\ndata_augmentation = False\nsave_dir = os.path.join(os.getcwd(), 'saved_models')\nmodel_name = 'keras_cnn_training.h5'\n"

In [4]:
f = h5py.File('/content/gdrive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5','r')
H = f['h_Estimated']
f = h5py.File('/content/gdrive/My Drive/CTW_2019/r_Position_CTW_Train.h5','r')
pos = f['r_Position']


In [5]:
print(H.shape)
H = np.array(H)
pos = np.array(pos)

(2, 924, 16, 17486)


In [7]:
data = H[:,:,:,:1000]
print(data.shape)
pos = pos[:,:1000]
print(pos.shape)

(2, 924, 16, 1000)
(3, 1000)


In [8]:
H = data[:,:,:,:900]
H = H.swapaxes(0, -1)
pos = pos[:,:900]
H_test = data[:,:,:,900:]
H_test = H_test.swapaxes(0, -1)
pos_test = pos[:,900:]
print(H.shape)
print(H_test.shape)

(900, 924, 16, 2)
(100, 924, 16, 2)


In [13]:
H1 = H[1]
H2 = H_test[1]

#print(H1[:,:,0].shape)

Yd1 = np.sum((np.multiply(H1[:,:,0], H1[:,:,0]) + np.multiply(H1[:,:,1], H1[:,:,1])), axis=0)
Yd2 = np.sum((np.multiply(H2[:,:,0], H2[:,:,0]) + np.multiply(H2[:,:,1], H2[:,:,1])), axis=0)
print(Yd2.shape)   #We have 16 links

G_re = np.multiply(H1[:,:,0], H2[:,:,0]) + np.multiply(H1[:,:,1], H2[:,:,1])
G_im = np.multiply(H2[:,:,0], H1[:,:,1]) - np.multiply(H1[:,:,0], H2[:,:,1])
G = G_re + G_im * 1j
print(G.shape)

Nser = 1024
G = np.concatenate((G, np.zeros((Nser-924, 16))), axis=0)

g = np.fft.fftn(G, axes=(0,))
print(g.shape)

phi_d = np.ndarray.max((np.square(np.abs(g))), axis=0)
phi_d = np.divide(phi_d,np.multiply(Yd1, Yd2))
print(phi_d.shape)

w_d = np.sqrt(np.multiply(Yd1,Yd2))/(np.sqrt(np.sum(Yd1)) * np.sum(Yd2))

TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)))
TRRS = TRRS**2

print(TRRS)

(16,)
(924, 16)
(1024, 16)
(16,)
0.02238512572668457
